In [1]:
%matplotlib inline

In [2]:
import sys
sys.path.append('/notebook/shared/extra')
from ggplot import *

In [3]:
import pandas as pd 
import math 
from impala.dbapi import connect
from impala.util import as_pandas
import re 

In [4]:
#get data
def get_data(sql):
    conn = connect(host = '10.1.169.3',port = 10000, auth_mechanism = 'PLAIN')
    cursor = conn.cursor()
    cursor.execute(sql)
    df = as_pandas(cursor)
    return df
def dropna_all(df):
    df_copy = df.copy()
    df_copy = df_copy.dropna(subset = ['phone_gray_score','honeypot_model_score','contacts_relatives_redline_cnt','contacts_loankeywords_redline_cnt','tongdun_score','n_91applytimes_cnt','n_91rejectprecent','ant_score','tc_loankeywords_callcnt_last1m','tc_collection_callcnt_last3m','tc_relatives_callcnt_lasthalfyear'],
                    how ='all')
    return df_copy
#get_everyday_num
def get_oneday_num(df,time):
    num =len( df[df['dt'] == time])
    return num
def get_everyday_num(df):
    nums =[]
    times = pd.date_range('20170201','20170520')
    for time in times:
        time = str(time).split(' ')[0]
        num = get_oneday_num(df,time)
        temp = [time,num]
        nums.append(temp)
    return nums
# caculate the mounts of nan
def missing_values_table(df): 
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum()/len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        return mis_val_table_ren_columns
def get_df_time(df,start,end):
    df_copy = df.copy()
    df_copy = df_copy[(df_copy.dt >=start) & (df_copy.dt <end)]
    return df_copy
def col_names():
    col_names = []
    #add time
    #col_names = ['dt']
    for i in range(1,18):
        col_name = "F"+str(i)
        col_names.append(col_name)
    return col_names
def get_Y_X(df):
    df_copy = df.copy()
    df_Y = df_copy['labels'].to_frame(name = 'Y')
    #del df_copy[['labels','aid','uid','dt']]
    #save dt
    #df_Y['Y'] = df_Y['Y'].map(convert_labels)
    df_copy.drop(df.columns[[0,1,2,3]],axis=1, inplace=True)
    df_X = df_copy
    df_X.columns = col_names()
    return df_Y,df_X
#map categorical variable
def map_X(df_x):
    return df_x
def drop_X(df_x):
    del df_x['F1']
    del df_x['F3']
    del df_x['F5']
    del df_x['F8']
    del df_x['F10']
    del df_x['F13']
    return df_x
def fillna(df):
    return df.fillna(-999)
def get_fix_X(df_x):
    #df_x = map_X(df_x)
    df_x = drop_X(df_x)
    #df_x = dummies_X_all(df_x)
    df_x = fillna(df_x)
    return df_x

In [5]:
df_all_train = get_data('select * from tmp.cebai_project1_datasource_02010520_labels_fix')
df_ar_train = get_data('select * from tmp.cebai_armodel_20170201_20170520_labels_fix')
df_ar_score_before = get_data('select * from tmp.cebai_armodel_20170201_20170520_fix ')
df_ar_score_now = get_data('select * from tmp.cebai_armodel_20170620_20170630_fix')

In [133]:
%store df_all_train
%store df_ar_train
%store df_ar_score_before
%store df_ar_score_now

Stored 'df_all_train' (DataFrame)
Stored 'df_ar_train' (DataFrame)
Stored 'df_ar_score_before' (DataFrame)
Stored 'df_ar_score_now' (DataFrame)


In [6]:
df_all_train_drop = dropna_all(df_all_train)
df_ar_train_drop = dropna_all(df_ar_train)
df_ar_score_before_drop = dropna_all(df_ar_score_before)
df_ar_score_now_drop = dropna_all(df_ar_score_now)
print(len(df_all_train),len(df_all_train_drop))
print(len(df_ar_train),len(df_ar_train_drop))
print(len(df_ar_score_before),len(df_ar_score_before_drop))
print(len(df_ar_score_now),len(df_ar_score_now_drop))

1037407 1036863
505307 505307
728507 728507
132947 132947


In [7]:
print(len(get_df_time(df_all_train_drop,'2017-02-01','2017-05-01')))
print(len(get_df_time(df_ar_train_drop,'2017-02-01','2017-05-01')))

661206
332572


In [84]:
df_reference_pass02 = get_data('select * from tmp.cebai_armodel_20170201_20170520_fix_reference_pass')
df_reference_pass06 = get_data('select * from tmp.cebai_armodel_20170620_20170630_fixreference_pass')

In [94]:
df_reference_pass02_drop = dropna_all(df_reference_pass02)
df_reference_pass06_drop = dropna_all(df_reference_pass06)
df_reference_pass02_drop.drop(['loan_status'],axis = 1 ,inplace = True)
df_reference_pass06_drop.drop(['loan_status'],axis = 1 ,inplace = True)
print(len(df_reference_pass02),len(df_reference_pass02_drop))
print(len(df_reference_pass06),len(df_reference_pass06_drop))

88216 88216
5269 5269


In [134]:
%store df_reference_pass02_drop
%store df_reference_pass06_drop

Stored 'df_reference_pass02_drop' (DataFrame)
Stored 'df_reference_pass06_drop' (DataFrame)


In [96]:
print(df_reference_pass06_drop.head(5))

      status        uid          dt       aid direct_contact_score  \
0  reference  109423270  2017-06-29  41432175                 None   
1  reference   93904428  2017-06-23  40055845                 None   
2  reference   93574164  2017-06-28  40771639                 None   
3  reference  116024537  2017-06-21  39604774                 None   
4  reference  103076874  2017-06-26  39544466                 None   

  phone_gray_score mob_score honeypot_model_score  \
0               79      None                  597   
1               20      None                  581   
2               91      None                  569   
3               -1      None                  624   
4               86      None                  622   

  contacts_relatives_redline_cnt2 contacts_relatives_redline_cnt  \
0                            None                             56   
1                            None                              3   
2                            None                       

In [1]:
missing_values_table(get_df_time(df_ar_train_drop,'2017-02-01','2017-04-01'))

NameError: name 'missing_values_table' is not defined

In [34]:
#RF_tarin
from scipy.stats import ks_2samp
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.metrics import roc_auc_score,roc_curve
import matplotlib.pyplot as plot
import numpy as np
from math import sqrt,fabs,exp
def train_online(df):
    df_online = get_df_time(df,'2017-02-01','2017-04-01')
    y,X = get_Y_X(df_online)
    X = get_fix_X(X)
    #y = normalize(y)
    #X = normalize(X)
    xTrain,xTest,yTrain,yTest = train_test_split(X.values,y.values.ravel(),test_size = 0.2,random_state = 520)
    rf = ensemble.RandomForestClassifier(n_estimators=2000,max_depth=7,oob_score = True,random_state = 531,class_weight = 'auto')
    rf.fit(xTrain,yTrain)
    print("online----------oob_score:%f" % rf.oob_score_)
    prediction = rf.predict_proba(xTest)
    auc = roc_auc_score(yTest,prediction[:,1:2])
    print ("online-------AUC Score (test):%f" % auc)
    return rf

In [20]:
#grid search
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn import cross_validation, metrics
def grid_search(df):
    df_online = get_df_time(df,'2017-02-01','2017-04-01')
    y,X = get_Y_X(df_online)
    X = get_fix_X(X)
    #y = normalize(y)
    #X = normalize(X)
    X,y  = X.values,y.values.ravel()
    
    param_test1 = {'n_estimators':np.linspace(10,200,50,dtype = int),'max_depth':np.linspace(1,11,9,dtype = int)}
    gsearch1 = GridSearchCV(estimator = RandomForestClassifier(random_state=531,n_jobs=-1), 
                       param_grid = param_test1, scoring='roc_auc',cv=5)
    gsearch1.fit(X,y)
    print(gsearch1.grid_scores_)
    print('***********************')
    print(gsearch1.best_params_)
    print('***********************')
    print(gsearch1.best_score_)

In [21]:
rf_fix = train_online(df_all_train_drop)#n_estimators=800,max_depth=7

online----------oob_score:0.545765
online-------AUC Score (test):0.642182


In [33]:
rf_ar_fix = train_online(delete_armodel_status(df_ar_train_drop))#1000,7

online----------oob_score:0.607806
online-------AUC Score (test):0.615998


In [35]:
rf_ar_fix_04 = train_online(delete_armodel_status(df_ar_train_drop))#2000 7

online----------oob_score:0.573893
online-------AUC Score (test):0.606060


In [129]:
%store -r est

In [36]:
import numpy as np
import pandas as pd
from collections import Counter
def ks_value(pred,true,n):
    score = (1.-pred)*1000
    index = score.argsort()
    score_sort = np.sort(score)
    true_sort = true[index]
    len_num = pred.size
    add = int(len_num/n)
    out = []
    for i in get_split_len(len_num,n):
        if i == get_split_len(len_num,n)[-1]:
            good_num,bad_num,total = get_good_bad(true_sort,i,-1)
            min_value,max_value = get_max_min(score_sort,i,-1)
            temp = [min_value,max_value,total,bad_num,good_num]
        else:
            good_num,bad_num,total = get_good_bad(true_sort,i,i+add)
            min_value,max_value = get_max_min(score_sort,i,i+add)
            temp = [min_value,max_value,total,bad_num,good_num]
        out.append(temp)
    return out
def list_to_dataframe(out):
    df = pd.DataFrame.from_records(out,columns = ["min_value","max_value","total","bad_num","good_num"])
    return df
def get_good_bad(true,startindex,endindex):
    if endindex == -1:
        true_index = true[startindex:]
    else:
        true_index = true[startindex:endindex]
    c = Counter([i for i in true_index])
    total = true_index.size
    return c[0],c[1],total
def get_max_min(pred,startindex,endindex):
    return pred[startindex],pred[endindex]
def get_split_len(len_num,n):
    out = [0]
    add = int(len_num/n)
    index = 0
    for i in range(n-1):
        index += add
        out.append(int(index))
    return out
def get_ksvalue_offline(rf,df,start,end,name):
    df_offline = get_df_time(df,start,end)
    y,X = get_Y_X(df_offline)
    X = get_fix_X(X)
    xTest,yTest = X.values,y.values.ravel()
    prediction = rf.predict_proba(xTest)
    pred = prediction[:,1:2].ravel()
    out = ks_value(pred,yTest,10)
    df = list_to_dataframe(out)
    df.to_csv(name)
    return df

In [50]:
def get_start_end(pred,cuts):
    out1 = cuts_off_array(pred,cuts)
    out2 = get_cuts_num(out1)
    out = [0]
    index = 0
    for i in out2:
        index += i
        out.append(int(index))
    return out
def ks_value_cuts(pred,true,cuts):
    score = (1.-pred)*1000
    index = score.argsort()
    score_sort = np.sort(score)
    true_sort = true[index]
    out = []
    start_end = get_start_end(score_sort,cuts)
    for i in range(10):
        if i == 9:
            good_num,bad_num,total = get_good_bad(true_sort,start_end[i],-1)
            min_value,max_value = get_max_min(score_sort,start_end[i],-1)
            temp = [min_value,max_value,total,bad_num,good_num]
        else:
            good_num,bad_num,total = get_good_bad(true_sort,start_end[i],start_end[i+1])
            min_value,max_value = get_max_min(score_sort,start_end[i],start_end[i+1])
            temp = [min_value,max_value,total,bad_num,good_num]
        out.append(temp)
    return out
def get_ksvalue_offline_cuts(rf,df,start,end,name,cuts):
    df_offline = get_df_time(df,start,end)
    y,X = get_Y_X(df_offline)
    X = get_fix_X(X)
    xTest,yTest = X.values,y.values.ravel()
    prediction = rf.predict_proba(xTest)
    pred = prediction[:,1:2].ravel()
    #fix
    out = ks_value_cuts(pred,yTest,cuts)
    df = list_to_dataframe(out)
    df.to_csv(name)
    return df

In [38]:
def get_ks_max(total,bad):
    sum_total = np.sum(total)
    sum_bad= np.sum(bad)
    ks = []
    for i in range(9):
        ks_temp = -(-(sum(total[:i+1])-sum(bad[:i+1]))/(sum_total-sum_bad)+sum(bad[:i+1])/sum_bad)
        ks.append(ks_temp)   
    max_ks = np.amax(ks)
    return max_ks

In [39]:
def delete_armodel_status(df):
    df_copy = df.copy()
    df_copy.drop(['status'],axis = 1 ,inplace = True)
    return df_copy

In [44]:
cuts=[401.24,434.55,455.51,474.88,507.85,545.93,582.47,612.77,671.79]
cuts1=[422.59,454.10,476.80,499.63,521.30,539.68,560.84,587.72,620.07]
cuts2 = [434.59,454.20,470.81,490.46,516.38,542.38,571.05,598.341,623.24]

In [40]:
get_ksvalue_offline(rf_ar_fix_04,delete_armodel_status(df_ar_train_drop),'2017-02-01','2017-04-01','02_ar_df1_fix.csv')
#get_ksvalue_offline(rf_ar_fix_04,delete_armodel_status(df_ar_train_drop),'2017-01-20','2017-01-30','02_ar_df5_fix.csv')

,min_value,max_value,total,bad_num,good_num
0,315.833258,434.598292,14793,2206,12587
1,434.598292,454.203674,14793,1515,13278
2,454.203674,470.812234,14793,1215,13578
3,470.812234,490.467824,14793,1077,13716
4,490.467824,516.380040,14793,990,13803
5,516.380040,542.382799,14793,864,13929
6,542.382799,571.059650,14793,760,14033
7,571.059650,598.341000,14793,666,14127
8,598.341000,623.245514,14793,505,14288
9,623.245514,820.666231,14796,297,14499


In [51]:
get_ksvalue_offline_cuts(rf_ar_fix_04,delete_armodel_status(df_ar_train_drop),'2017-04-01','2017-04-06','02_ar_df2_fix.csv',cuts2)
get_ksvalue_offline_cuts(rf_ar_fix_04,delete_armodel_status(df_ar_train_drop),'2017-05-01','2017-05-10','02_ar_df3_fix.csv',cuts2)
get_ksvalue_offline_cuts(rf_ar_fix_04,delete_armodel_status(df_ar_train_drop),'2017-05-10','2017-05-20','02_ar_df4_fix.csv',cuts2)

,min_value,max_value,total,bad_num,good_num
0,321.439312,434.599019,5451,1354,4097
1,434.599019,454.201027,5969,1205,4764
2,454.201027,470.811138,6319,1111,5208
3,470.811138,490.463020,7079,1090,5989
4,490.463020,516.380933,8909,1208,7701
5,516.380933,542.380890,9941,1137,8804
6,542.380890,571.054755,12086,1262,10824
7,571.054755,598.341040,11200,1003,10197
8,598.341040,623.241628,8921,718,8203
9,623.241628,800.550444,11461,677,10784


In [41]:
get_ksvalue_offline(rf_ar_fix,delete_armodel_status(df_ar_train_drop),'2017-02-01','2017-05-01','ar_df1_fix.csv')
#get_ksvalue_offline(rf_ar_fix,delete_armodel_status(df_ar_train_drop),'2017-01-20','2017-01-30','ar_df5_fix.csv')

,min_value,max_value,total,bad_num,good_num
0,290.510299,422.598802,33257,5551,27706
1,422.598802,454.104862,33257,3829,29428
2,454.104862,476.800587,33257,3082,30175
3,476.800587,499.639771,33257,2703,30554
4,499.639771,521.300286,33257,2429,30828
5,521.300286,539.688766,33257,2222,31035
6,539.688766,560.847836,33257,2029,31228
7,560.847836,587.729913,33257,1783,31474
8,587.729913,620.072435,33257,1460,31797
9,620.072435,796.292786,33259,1039,32220


In [52]:
get_ksvalue_offline_cuts(rf_ar_fix,delete_armodel_status(df_ar_train_drop),'2017-05-01','2017-05-05','ar_df2_fix.csv',cuts1)
get_ksvalue_offline_cuts(rf_ar_fix,delete_armodel_status(df_ar_train_drop),'2017-05-05','2017-05-10','ar_df3_fix.csv',cuts1)
get_ksvalue_offline_cuts(rf_ar_fix,delete_armodel_status(df_ar_train_drop),'2017-05-10','2017-05-20','ar_df4_fix.csv',cuts1)

,min_value,max_value,total,bad_num,good_num
0,300.103403,422.593029,6772,1583,5189
1,422.593029,454.100887,6726,1290,5436
2,454.100887,476.810376,6988,1161,5827
3,476.810376,499.633558,7288,1066,6222
4,499.633558,521.300317,8471,1087,7384
5,521.300317,539.681642,7356,815,6541
6,539.681642,560.842210,9142,959,8183
7,560.842210,587.720506,12446,1197,11249
8,587.720506,620.072748,11199,950,10249
9,620.072748,799.122110,10948,657,10291


In [43]:
get_ksvalue_offline(rf_fix,df_all_train_drop,'2017-02-01','2017-05-01','df1_fix.csv')
#get_ksvalue_offline(rf_fix,delete_armodel_status(df_ar_train_drop),'2017-01-20','2017-01-30','df5_fix.csv')

,min_value,max_value,total,bad_num,good_num
0,290.632158,401.247038,66120,9025,57095
1,401.247038,434.559459,66120,6811,59309
2,434.559459,455.519460,66120,5826,60294
3,455.519460,474.884347,66120,4830,61290
4,474.884347,507.852347,66120,4202,61918
5,507.852347,545.932380,66120,3864,62256
6,545.932380,582.474547,66120,3240,62880
7,582.474547,612.775829,66120,2648,63472
8,612.775829,671.796982,66120,2146,63974
9,671.796982,860.719127,66126,1428,64698


In [53]:
get_ksvalue_offline_cuts(rf_fix,delete_armodel_status(df_ar_train_drop),'2017-05-01','2017-05-05','df2_fix.csv',cuts)
get_ksvalue_offline_cuts(rf_fix,delete_armodel_status(df_ar_train_drop),'2017-05-05','2017-05-10','df3_fix.csv',cuts)
get_ksvalue_offline_cuts(rf_fix,delete_armodel_status(df_ar_train_drop),'2017-05-10','2017-05-20','df4_fix.csv',cuts)

,min_value,max_value,total,bad_num,good_num
0,299.660360,401.240642,9614,2030,7584
1,401.240642,434.556858,9974,1776,8198
2,434.556858,455.510987,8820,1367,7453
3,455.510987,474.881623,6629,857,5772
4,474.881623,507.856655,10521,1249,9272
5,507.856655,545.931372,13758,1443,12315
6,545.931372,582.476434,12442,1112,11330
7,582.476434,612.773282,8718,647,8071
8,612.773282,671.795383,6048,256,5792
9,671.795383,751.088516,812,28,784


In [83]:
def select_status(df,status):
    df_copy = df.copy()
    if status != 'all':
        df_copy = df_copy[df_copy.status == status]
    df_copy.drop(df_copy.columns[[0]],axis = 1 ,inplace = True)
    return df_copy
def get_pred_armodel(model,df,status,start,end):
    df = get_df_time(df,start,end)
    df = select_status(df,status)
    df.drop(df.columns[[0,1,2]],axis=1, inplace=True)
    df.columns = col_names() 
    X = get_fix_X(df)
    xTest = X.values
    prediction = model.predict_proba(xTest)
    csvname = 'pred_'+status+'.csv'
    score = (1.-prediction)*1000
    #pd.DataFrame.from_records(score,columns = ['F0','F1']).to_csv(csvname,index = False)
    pred = score[:,1:2].ravel()
    return pred
def get_pred_train(rf,df,start,end):
    df_offline = get_df_time(df,start,end)
    y,X = get_Y_X(df_offline)
    X = get_fix_X(X)
    xTest = X.values
    prediction = rf.predict_proba(xTest)
    score = (1.-prediction)*1000
    pred = score[:,1:2].ravel()
    return pred
def get_pred_armodel_reference_pass(model,df,start,end):
    df = get_df_time(df,start,end)
    df.drop(df.columns[[0,1]],axis = 1 ,inplace = True)
    df.drop(df.columns[[0,1,2]],axis=1, inplace=True)
    df.columns = col_names() 
    X = get_fix_X(df)
    xTest = X.values
    prediction = model.predict_proba(xTest)
    csvname = 'pred_'+status+'.csv'
    score = (1.-prediction)*1000
    #pd.DataFrame.from_records(score,columns = ['F0','F1']).to_csv(csvname,index = False)
    pred = score[:,1:2].ravel()
    return pred

In [56]:
def get_score_and_x(rf,df,start,end):
    df_offline = get_df_time(df,start,end)
    y,X = get_Y_X(df_offline)
    X = get_fix_X(X)
    X = X.reset_index(drop=True)
    xTest = X.values
    prediction = rf.predict_proba(xTest)
    score = (1.-prediction)*1000
    score = score[:,1:2]
    score = pd.DataFrame(score,columns  = [['pred']])
    result = pd.concat([score,X],axis=1)
    return result

In [57]:
def cuts_off_array(array,cuts):
    array.sort()
    out =[]
    list1,list2,list3,list4,list5,list6,list7,list8,list9,list10 = [],[],[],[],[],[],[],[],[],[]
    for i in array:
        if i <=cuts[0]:
            list1.append(i)
        elif i>cuts[0] and i <= cuts[1]:
            list2.append(i)
        elif i>cuts[1] and i <=cuts[2]:
            list3.append(i)
        elif i>cuts[2] and i <=cuts[3]:
            list4.append(i)
        elif i>cuts[3] and i <= cuts[4]:
            list5.append(i)
        elif i > cuts[4] and i <= cuts[5]:
            list6.append(i)
        elif i>cuts[5] and i <= cuts[6]:
            list7.append(i)
        elif i>cuts[6] and i <= cuts[7]:
            list8.append(i)
        elif i>cuts[7] and i <= cuts[8]:
            list9.append(i)
        elif i>cuts[8]:
            list10.append(i)
    out.append(list1)  
    out.append(list2)  
    out.append(list3)  
    out.append(list4)  
    out.append(list5)  
    out.append(list6)  
    out.append(list7)  
    out.append(list8)
    out.append(list9)  
    out.append(list10)
    return out
def get_cuts_num(cuts):
    out =[]
    for i in cuts:
        out.append(len(i))
    return out
def get_cuts_mean(cuts):
    out = []
    for i in cuts:
        if len(i) == 0:
            out.append(0.0)
        else:
            out.append(sum(i) / float(len(i)))
    return out

In [58]:
def get_sub():
    big_aid = df_drop['aid'].tolist()
    small_aid = df_armodel_labels_drop['aid'].tolist()
    sub_set = set(big_aid) - set(small_aid)
    sub = [i for i in sub_set]
    df_drop_sub = df_drop[df_drop['aid'].isin(sub)]

In [59]:
train_score = get_pred_train(rf_fix,df_all_train_drop,'2017-02-01','2017-05-01')
print(get_cuts_num(cuts_off_array(train_score,cuts)))
print(get_cuts_mean(cuts_off_array(train_score,cuts)))

[66110, 66099, 66115, 66137, 66133, 66122, 66111, 66128, 66123, 66128]
[371.90361258409905, 419.75040765269222, 445.5494412595271, 465.25611345965717, 490.60041842570507, 525.5673571335858, 565.24079487132246, 596.69703811114391, 640.00364215235118, 728.48154301851923]


In [82]:
armodel_reject_score_fix = get_pred_armodel(rf_fix,df_ar_score_before_drop,'reject','2017-05-01','2017-05-10')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix,cuts)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix,cuts)))

[1478, 3106, 1424, 566, 731, 675, 573, 206, 59, 5]
[376.49541434452425, 421.09174225984771, 444.29273815360165, 463.80589233276606, 492.1830776463994, 526.19145732136883, 565.17103988699068, 593.33894512073971, 636.38005260720706, 678.61445588188099]


In [110]:
armodel_reject_score_fix = get_pred_armodel(rf_fix,df_ar_score_before_drop,'pass','2017-05-01','2017-05-10')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix,cuts)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix,cuts)))

[13590, 11845, 10490, 7477, 10680, 11593, 11311, 8076, 5892, 582]
[368.59676225953092, 420.0047865705119, 445.09964397343884, 464.45455572510951, 491.90687377279812, 526.59189470715182, 564.76444368663829, 595.91115700731439, 633.90675508401773, 686.41657640336734]


In [60]:
armodel_reject_score_fix = get_pred_armodel(rf_fix,df_ar_score_before_drop,'all','2017-05-01','2017-05-10')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix,cuts)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix,cuts)))

[19612, 17843, 13740, 9164, 12727, 13405, 12809, 8534, 6067, 597]
[368.48223028774385, 420.05842636275452, 444.95021363980118, 464.4294720101592, 491.87296018614296, 526.5105406133282, 564.73919328881971, 595.74356678102936, 633.95603698395075, 686.28002022300814]


In [109]:
armodel_reject_score_fix = get_pred_armodel(rf_fix,df_ar_score_before_drop,'reference','2017-05-01','2017-05-10')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix,cuts)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix,cuts)))

[4544, 2892, 1826, 1121, 1316, 1137, 925, 252, 116, 10]
[365.53329201034302, 419.16834481458875, 444.60449671346817, 464.5770145264184, 491.42547312768744, 525.8704730780247, 564.16291723424547, 592.33838183869045, 635.22631312736382, 682.16523269665902]


In [98]:
armodel_reject_score_fix = get_pred_armodel(rf_fix,df_reference_pass02_drop,'all','2017-05-01','2017-05-10')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix,cuts)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix,cuts)))

[2039, 1482, 1000, 594, 702, 580, 460, 129, 49, 7]
[367.67631577239331, 419.04803269115797, 444.64700047479761, 464.36899160278875, 491.69319816867039, 525.82936304311875, 564.39591534201452, 593.18674654571237, 634.16225906058344, 684.024245900035]


In [61]:
armodel_reject_score_fix_0510 = get_pred_armodel(rf_fix,df_ar_score_before_drop,'reject','2017-05-10','2017-05-20')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_0510,cuts)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_0510,cuts)))

[1034, 3061, 1559, 671, 923, 1157, 824, 308, 96, 4]
[377.96109272142741, 421.5602435009352, 444.43706550964544, 463.96647708082628, 493.23794431841378, 528.41539637782546, 562.50588678086513, 593.24606880620479, 637.36519856258985, 682.6491644140749]


In [62]:
armodel_reject_score_fix_0510 = get_pred_armodel(rf_fix,df_ar_score_before_drop,'all','2017-05-10','2017-05-20')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_0510,cuts)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_0510,cuts)))

[15175, 15673, 12325, 8729, 13691, 17947, 15744, 10751, 7329, 1027]
[368.34942038133721, 420.06674253730432, 444.83981385115646, 464.75894705281291, 492.08362084193578, 528.60825853979986, 563.45320892908444, 595.62342630422427, 636.2085782877059, 686.52497768751311]


In [111]:
armodel_reject_score_fix_0510 = get_pred_armodel(rf_fix,df_ar_score_before_drop,'pass','2017-05-10','2017-05-20')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_0510,cuts)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_0510,cuts)))

[10558, 9935, 8850, 6831, 10970, 14736, 13646, 10004, 7055, 1010]
[368.04464273380944, 419.80462042047293, 445.00204953875681, 464.85270463521289, 492.02064530528037, 528.6933199980715, 563.521924768811, 595.8137545470986, 636.17123819809649, 686.60117678901099]


In [113]:
armodel_reject_score_fix_0510 = get_pred_armodel(rf_fix,df_ar_score_before_drop,'reference','2017-05-10','2017-05-20')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_0510,cuts)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_0510,cuts)))

[3583, 2677, 1916, 1227, 1798, 2054, 1274, 439, 178, 13]
[366.47372214046771, 419.33180670653138, 444.41815353234574, 464.67034828005796, 491.87527883321826, 528.10664017188094, 563.32989425187384, 592.95413784792538, 637.06475124165513, 681.79745157835782]


In [99]:
armodel_reject_score_fix_0510 = get_pred_armodel(rf_fix,df_reference_pass02_drop,'all','2017-05-10','2017-05-20')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_0510,cuts)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_0510,cuts)))

[1624, 1313, 978, 639, 925, 1012, 658, 238, 99, 8]
[367.50639217354603, 419.55111026008069, 444.34409911440827, 464.66552264755973, 491.47919034449893, 527.82330414300679, 562.99730120147069, 593.17619167310784, 637.345507257794, 684.5586230538978]


In [63]:
armodel_reject_score_fix_06 = get_pred_armodel(rf_fix,df_ar_score_now_drop,'reject','2017-06-20','2017-06-30')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_06,cuts)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_06,cuts)))

[769, 1265, 943, 545, 634, 602, 572, 160, 69, 2]
[382.33388005240101, 419.63764553204197, 444.01531347018323, 464.03145056826168, 492.17703633369541, 527.82966708997583, 566.80714452122868, 592.22435553884122, 633.62024813076641, 682.77648823061679]


In [64]:
armodel_reject_score_fix_06 = get_pred_armodel(rf_fix,df_ar_score_now_drop,'all','2017-06-20','2017-06-30')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_06,cuts)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_06,cuts)))

[13938, 17221, 15906, 11819, 16394, 17222, 17152, 12040, 10301, 954]
[375.6316691394988, 419.9651240847806, 444.65677247771197, 464.2604709945079, 492.28870756693249, 526.52598615617524, 564.9400372794131, 596.1687603649633, 633.79564036418367, 685.13625429313493]


In [114]:
armodel_reject_score_fix_06 = get_pred_armodel(rf_fix,df_ar_score_now_drop,'pass','2017-06-20','2017-06-30')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_06,cuts)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_06,cuts)))

[10557, 13249, 12937, 10052, 14226, 15145, 15308, 11563, 10111, 940]
[375.39340126907518, 420.07052494842424, 444.73887937418351, 464.30426994654096, 492.31761654543311, 526.44644749887993, 564.91342965993329, 596.30016176030597, 633.80376358236833, 685.19074256034037]


In [115]:
armodel_reject_score_fix_06 = get_pred_armodel(rf_fix,df_ar_score_now_drop,'reference','2017-06-20','2017-06-30')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_06,cuts)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_06,cuts)))

[2612, 2707, 2026, 1222, 1534, 1475, 1272, 317, 121, 12]
[374.62148296647206, 419.60228859411961, 444.43104637921329, 464.00232784105123, 492.06676521601753, 526.81059432063603, 564.42063798432355, 593.36658508997789, 633.21686519923708, 681.26130103909748]


In [97]:
armodel_reject_score_fix_06 = get_pred_armodel(rf_fix,df_reference_pass06_drop,'all','2017-06-20','2017-06-30')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_06,cuts)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_06,cuts)))

[968, 1040, 838, 481, 626, 586, 523, 143, 57, 7]
[375.68189829737128, 419.75240801050603, 444.36509445919148, 463.95082571734554, 492.01544489824124, 526.93345771754446, 564.40141911979231, 592.47370285349257, 633.88215684554939, 682.39859644151409]


In [65]:
train_score = get_pred_train(rf_ar_fix,delete_armodel_status(df_ar_train_drop),'2017-02-01','2017-05-01')
print(get_cuts_num(cuts_off_array(train_score,cuts1)))
print(get_cuts_mean(cuts_off_array(train_score,cuts1)))

[33252, 33252, 33266, 33244, 33271, 33238, 33268, 33253, 33266, 33262]
[386.19194040698932, 439.8961280724925, 465.46678418241601, 488.32818100015533, 510.83475535597745, 530.69482976108736, 549.70160810591403, 573.43497827095109, 603.80995648545229, 644.51422459952767]


In [66]:
armodel_reject_score_fix = get_pred_armodel(rf_ar_fix,df_ar_score_before_drop,'reject','2017-05-01','2017-05-10')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix,cuts1)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix,cuts1)))

[3104, 1819, 896, 684, 646, 492, 371, 313, 379, 119]
[389.75975499003556, 437.26898615073321, 464.58959448345314, 487.73093148380383, 510.98242093289997, 530.29244594154557, 550.51440885970123, 571.69663195784369, 603.17549337371861, 642.04513384393397]


In [116]:
armodel_reject_score_fix = get_pred_armodel(rf_ar_fix,df_ar_score_before_drop,'pass','2017-05-01','2017-05-10')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix,cuts1)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix,cuts1)))

[10906, 8963, 7706, 8179, 9328, 8025, 8759, 9101, 10451, 10118]
[385.21096549253951, 439.26438489805139, 465.30685368272754, 488.70374029690294, 510.75526518909612, 530.57441589966368, 550.28299934867459, 573.22134430176982, 604.09096091721335, 649.12981553531699]


In [117]:
armodel_reject_score_fix = get_pred_armodel(rf_ar_fix,df_ar_score_before_drop,'reference','2017-05-01','2017-05-10')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix,cuts1)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix,cuts1)))

[3795, 2345, 1600, 1452, 1324, 1005, 842, 702, 777, 297]
[382.59793843386592, 438.4341699542282, 464.7831683228523, 488.29727363103501, 510.57853710701386, 530.32690240602699, 550.46635625748638, 572.21665229589826, 603.14371002236862, 643.31854641944881]


In [67]:
armodel_reject_score_fix = get_pred_armodel(rf_ar_fix,df_ar_score_before_drop,'all','2017-05-01','2017-05-10')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix,cuts1)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix,cuts1)))

[17805, 13127, 10202, 10315, 11298, 9522, 9972, 10116, 11607, 10534]
[385.44702305572616, 438.83957463183691, 465.16172911711772, 488.58201554392002, 510.74754299311616, 530.53372273851198, 550.30709074917752, 573.10444743029893, 603.99765720873768, 648.88593628254762]


In [100]:
armodel_reject_score_fix = get_pred_armodel(rf_ar_fix,df_reference_pass02_drop,'all','2017-05-01','2017-05-10')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix,cuts1)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix,cuts1)))

[1704, 1182, 819, 742, 710, 528, 451, 379, 382, 145]
[384.17109274018696, 438.50203971108465, 464.9342748304262, 488.3470326843717, 510.63931602090696, 530.66387397578239, 550.11119149743536, 572.4650920196583, 603.49899166867283, 642.57253547974699]


In [68]:
armodel_reject_score_fix_0510 = get_pred_armodel(rf_ar_fix,df_ar_score_before_drop,'reject','2017-05-10','2017-05-20')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_0510,cuts1)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_0510,cuts1)))

[2385, 2008, 1141, 818, 756, 589, 664, 622, 487, 167]
[392.67955463028483, 437.86792027217393, 464.65415720573651, 488.00332938495472, 511.22283126975094, 529.97364620091605, 551.36320555731004, 570.32131819629558, 604.38435577930863, 642.86375232981868]


In [118]:
armodel_reject_score_fix_0510 = get_pred_armodel(rf_ar_fix,df_ar_score_before_drop,'reference','2017-05-10','2017-05-20')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_0510,cuts1)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_0510,cuts1)))

[2707, 1989, 1801, 1604, 1632, 1217, 1353, 1436, 975, 445]
[383.5713881028413, 439.38666534249359, 465.65872665030122, 488.21655522344349, 510.4876185468159, 530.641313660121, 551.29044534146908, 571.6340939220496, 603.34172829679233, 641.78012296225222]


In [119]:
armodel_reject_score_fix_0510 = get_pred_armodel(rf_ar_fix,df_ar_score_before_drop,'pass','2017-05-10','2017-05-20')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_0510,cuts1)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_0510,cuts1)))

[7538, 6735, 7060, 7434, 8709, 7635, 9702, 13631, 12533, 12618]
[385.27395279802653, 439.8219134508393, 465.52735241650339, 488.63174410094024, 510.81150116686382, 530.64039969585826, 550.79197551243044, 573.15545080279037, 604.03665346446894, 650.43409240972471]


In [69]:
armodel_reject_score_fix_0510 = get_pred_armodel(rf_ar_fix,df_ar_score_before_drop,'all','2017-05-10','2017-05-20')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_0510,cuts1)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_0510,cuts1)))

[12630, 10732, 10002, 9856, 11097, 9441, 11719, 15689, 13995, 13230]
[386.30748547736681, 439.37564744354785, 465.45139653363788, 488.51201944619231, 510.79189128328085, 530.59892045488709, 550.88189158278317, 572.90384146108806, 604.00033899421874, 650.04745120054224]


In [101]:
armodel_reject_score_fix_0510 = get_pred_armodel(rf_ar_fix,df_reference_pass02_drop,'all','2017-05-10','2017-05-20')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_0510,cuts1)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_0510,cuts1)))

[1257, 938, 849, 787, 836, 645, 692, 743, 496, 251]
[384.99202341780085, 439.49589307404824, 465.51266632106331, 488.25826782573853, 510.37426095776266, 530.50290921272017, 550.92740375212861, 572.05367387055435, 603.24944506734323, 642.26578158944483]


In [70]:
armodel_reject_score_fix_06 = get_pred_armodel(rf_ar_fix,df_ar_score_now_drop,'reject','2017-06-20','2017-06-30')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_06,cuts1)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_06,cuts1)))

[978, 1116, 705, 517, 542, 446, 364, 320, 436, 137]
[394.95455260471067, 438.33428871781803, 464.32510492517332, 487.78213695309876, 511.18573332756648, 530.22104708908637, 550.96202039674483, 570.63402718749069, 604.79579279104985, 640.82531499314075]


In [121]:
armodel_reject_score_fix_06 = get_pred_armodel(rf_ar_fix,df_ar_score_now_drop,'reference','2017-06-20','2017-06-30')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_06,cuts1)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_06,cuts1)))

[2370, 2204, 1475, 1299, 1463, 1118, 1025, 898, 1070, 376]
[390.93173882282343, 438.85513867464056, 464.7339894871144, 488.2566404698581, 511.06411195355605, 530.93444718883359, 550.24135935045365, 572.27586411324819, 604.24356402230967, 641.95731345815318]


In [120]:
armodel_reject_score_fix_06 = get_pred_armodel(rf_ar_fix,df_ar_score_now_drop,'pass','2017-06-20','2017-06-30')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_06,cuts1)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_06,cuts1)))

[9130, 10903, 9238, 9027, 10614, 10217, 11340, 12834, 14392, 16393]
[391.70815664470268, 439.4880962894033, 465.08129144659563, 488.59612747141171, 511.08677276398333, 530.86469834726347, 550.19617737202532, 573.38395701721379, 604.52176966642116, 649.96911699819918]


In [71]:
armodel_reject_score_fix_06 = get_pred_armodel(rf_ar_fix,df_ar_score_now_drop,'all','2017-06-20','2017-06-30')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_06,cuts1)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_06,cuts1)))

[12478, 14223, 11418, 10843, 12619, 11781, 12729, 14052, 15898, 16906]
[391.81513412595143, 439.299480116106, 464.98973584247517, 488.51664515904707, 511.08839601937586, 530.84695034146785, 550.22171577951269, 573.25052085344726, 604.51056033462703, 649.71683147792601]


In [102]:
armodel_reject_score_fix_06 = get_pred_armodel(rf_ar_fix,df_reference_pass06_drop,'all','2017-06-20','2017-06-30')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_06,cuts1)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_06,cuts1)))

[897, 865, 546, 491, 574, 452, 430, 401, 451, 162]
[391.89726472591593, 438.57992476315667, 464.60942262996298, 488.61583596084614, 511.2917935178283, 531.06737612410257, 549.79618691071425, 572.52535838464814, 604.40998082742908, 644.5743154272219]


In [72]:
train_score = get_pred_train(rf_ar_fix_04,delete_armodel_status(df_ar_train_drop),'2017-02-01','2017-04-01')
print(get_cuts_num(cuts_off_array(train_score,cuts2)))
print(get_cuts_mean(cuts_off_array(train_score,cuts2)))

[14789, 14794, 14794, 14792, 14796, 14788, 14794, 14798, 14787, 14801]
[410.39090853533969, 445.12100039820672, 462.51325332235325, 480.07395180174956, 503.09816081934838, 529.30740154569821, 556.70346415068616, 585.04673456404987, 610.26111872261515, 658.69744617394701]


In [103]:
armodel_reject_score_fix_04 = get_pred_armodel(rf_ar_fix_04,df_ar_score_before_drop,'reject','2017-05-01','2017-05-10')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_04,cuts2)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_04,cuts2)))

[1732, 1603, 1357, 1156, 890, 637, 480, 399, 317, 252]
[415.02597794102809, 444.8331730692708, 462.12068330969339, 479.68566757144362, 502.67139374399989, 528.9038615933091, 555.49503927470346, 583.12905262856816, 610.04416023883027, 650.18586051878685]


In [122]:
armodel_reject_score_fix_04 = get_pred_armodel(rf_ar_fix_04,df_ar_score_before_drop,'reference','2017-05-01','2017-05-10')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_04,cuts2)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_04,cuts2)))

[2802, 2261, 1771, 1613, 1539, 1301, 1026, 827, 554, 445]
[411.43938343055783, 444.47325138775312, 462.57544416513014, 480.26223762674567, 502.86882719268374, 528.70168564358437, 555.98743232542404, 584.22859267277283, 610.02026837322614, 650.93639313366316]


In [123]:
armodel_reject_score_fix_04 = get_pred_armodel(rf_ar_fix_04,df_ar_score_before_drop,'pass','2017-05-01','2017-05-10')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_04,cuts2)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_04,cuts2)))

[7578, 7798, 8014, 8814, 10281, 9909, 10524, 10077, 8710, 9831]
[412.30268214383079, 444.91221836439013, 462.45978703545512, 480.35136978355183, 503.34123081923423, 529.29168524519605, 556.71607293035629, 584.91117218755574, 610.1843936555756, 657.52186523224054]


In [104]:
armodel_reject_score_fix_04 = get_pred_armodel(rf_ar_fix_04,df_ar_score_before_drop,'all','2017-05-01','2017-05-10')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_04,cuts2)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_04,cuts2)))

[12112, 11662, 11142, 11583, 12710, 11847, 12030, 11303, 9581, 10528]
[412.49239361395564, 444.81624735236289, 462.43687059503151, 480.27251957841384, 503.2371250617046, 529.20604051252417, 556.60520996981029, 584.79832082395774, 610.17026366919242, 657.06791307877813]


In [105]:
armodel_reject_score_fix_04 = get_pred_armodel(rf_ar_fix_04,df_reference_pass02_drop,'all','2017-05-01','2017-05-10')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_04,cuts2)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_04,cuts2)))

[1302, 1122, 877, 816, 810, 676, 522, 425, 272, 220]
[412.36693066237535, 444.4639935187509, 462.54417648949493, 480.52008114736816, 502.90899570466235, 528.66353703545008, 556.10741616822463, 583.94844980206085, 609.91720590280227, 650.75161309165014]


In [75]:
armodel_reject_score_fix_0510_04 = get_pred_armodel(rf_ar_fix_04,df_ar_score_before_drop,'reject','2017-05-10','2017-05-20')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_0510_04,cuts2)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_0510_04,cuts2)))

[1180, 1461, 1418, 1294, 1112, 894, 875, 682, 377, 344]
[415.09760496906648, 445.25168048029667, 462.47415080282889, 480.16218644025861, 503.125758078535, 529.31494437811841, 557.06322049232358, 583.43694463608972, 610.27336540751389, 652.32338702268146]


In [124]:
armodel_reject_score_fix_0510_04 = get_pred_armodel(rf_ar_fix_04,df_ar_score_before_drop,'reference','2017-05-10','2017-05-20')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_0510_04,cuts2)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_0510_04,cuts2)))

[2070, 1856, 1706, 1587, 1775, 1673, 1807, 1223, 775, 687]
[411.62496271296652, 444.94226876824592, 462.57016752651106, 480.13731161093312, 503.20675840759964, 529.4579521474933, 556.29241486211276, 583.53597162641915, 609.92689435413388, 654.18675805987959]


In [125]:
armodel_reject_score_fix_0510_04 = get_pred_armodel(rf_ar_fix_04,df_ar_score_before_drop,'pass','2017-05-10','2017-05-20')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_0510_04,cuts2)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_0510_04,cuts2)))

[5472, 6001, 6366, 7387, 9382, 10459, 12934, 12404, 9971, 13219]
[412.9257223746722, 445.11755031366931, 462.58816707621941, 480.43308634664788, 503.46908915544901, 529.78206725439293, 557.17170534489378, 584.50128387375992, 610.56491996426701, 660.11591424373648]


In [76]:
armodel_reject_score_fix_0510_04 = get_pred_armodel(rf_ar_fix_04,df_ar_score_before_drop,'all','2017-05-10','2017-05-20')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_0510_04,cuts2)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_0510_04,cuts2)))

[8722, 9318, 9490, 10268, 12269, 13026, 15616, 14309, 11123, 14250]
[412.9108460804336, 445.10366768060663, 462.56789496795142, 480.35323253047682, 503.40001904093572, 529.70837982730825, 557.06387989994153, 584.36804910972251, 610.51058337200357, 659.6419521621666]


In [106]:
armodel_reject_score_fix_0510_04 = get_pred_armodel(rf_ar_fix_04,df_reference_pass02_drop,'all','2017-05-10','2017-05-20')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_0510_04,cuts2)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_0510_04,cuts2)))

[954, 924, 823, 793, 886, 835, 890, 613, 426, 350]
[411.8628035895282, 444.93552657415921, 462.37920266918582, 479.97486781234301, 503.4321430404438, 529.36815134818903, 556.52424239330526, 583.75557908107146, 610.28370823322939, 655.94357141652006]


In [77]:
armodel_reject_score_fix_06_04 = get_pred_armodel(rf_ar_fix_04,df_ar_score_now_drop,'reject','2017-06-20','2017-06-30')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_06_04,cuts2)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_06_04,cuts2)))

[765, 849, 725, 657, 642, 586, 437, 352, 322, 226]
[417.73092004175504, 444.56660018478919, 462.28650465316349, 479.72859551242794, 503.22353957519374, 528.75588704276356, 556.47666650111648, 584.3474496827447, 609.34339717485307, 651.9563971722547]


In [127]:
armodel_reject_score_fix_06_04 = get_pred_armodel(rf_ar_fix_04,df_ar_score_now_drop,'reference','2017-06-20','2017-06-30')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_06_04,cuts2)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_06_04,cuts2)))

[2098, 1756, 1626, 1534, 1453, 1296, 1257, 1018, 743, 517]
[413.62669854207115, 444.66317667190748, 462.13610144792739, 480.08882100030593, 503.38272377173951, 529.1509874131682, 556.32718581043275, 585.08896299479875, 608.87827349942654, 650.97303230146599]


In [126]:
armodel_reject_score_fix_06_04 = get_pred_armodel(rf_ar_fix_04,df_ar_score_now_drop,'pass','2017-06-20','2017-06-30')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_06_04,cuts2)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_06_04,cuts2)))

[7119, 7944, 8524, 10430, 12814, 12307, 13494, 14067, 12370, 15019]
[414.05043236129995, 444.89089652351106, 462.59656687333813, 480.51963975709174, 503.3566073830836, 529.42454947060378, 556.69473355080891, 584.94461019470452, 610.14455734546561, 657.34047332941111]


In [78]:
armodel_reject_score_fix_06_04 = get_pred_armodel(rf_ar_fix_04,df_ar_score_now_drop,'all','2017-06-20','2017-06-30')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_06_04,cuts2)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_06_04,cuts2)))

[9982, 10549, 10875, 12621, 14909, 14189, 15188, 15437, 13435, 15762]
[414.24343772323272, 444.82689011048643, 462.50704853850124, 480.42609787913909, 503.35342256720457, 529.37194727107044, 556.65803992357871, 584.94051293813038, 610.05532604866323, 657.05442027631182]


In [108]:
armodel_reject_score_fix_06_04 = get_pred_armodel(rf_ar_fix_04,df_reference_pass06_drop,'all','2017-06-20','2017-06-30')
print(get_cuts_num(cuts_off_array(armodel_reject_score_fix_06_04,cuts2)))
print(get_cuts_mean(cuts_off_array(armodel_reject_score_fix_06_04,cuts2)))

[790, 686, 649, 635, 545, 521, 487, 437, 316, 203]
[413.85357295054126, 444.93473039622478, 462.09032681264404, 480.02000670496369, 503.34305832634033, 529.17261972166682, 556.00080106591361, 585.2259775900767, 609.36393265937943, 652.28530878741572]
